In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import flopy

In [ ]:
origin = [4.5, 2.3]
#origin = [0.0, 0.0]

In [ ]:
def f(x):
    '''Objective function'''
    return 0.5*(x[0] - origin[0])**2 + 2.5*(x[1] - origin[1])**2

def df(x):
    '''Gradient of the objective function'''
    return np.array([x[0] - origin[0], 5*(x[1] - origin[1])])

In [ ]:
def steepest_descent(gradient, x0 = np.zeros(2), alpha = 0.01, max_iter = 10000, tolerance = 1e-10): 
    '''
    Steepest descent with constant step size alpha.

    Args:
      - gradient: gradient of the objective function
      - alpha: line search parameter (default: 0.01)
      - x0: initial guess for x_0 and x_1 (default values: zero) <numpy.ndarray>
      - max_iter: maximum number of iterations (default: 10000)
      - tolerance: minimum gradient magnitude at which the algorithm stops (default: 1e-10)

    Out:
      - results: <numpy.ndarray> of size (n_iter, 2) with x_0 and x_1 values at each iteration
      - number of steps: <int>
    '''

    # Prepare list to store results at each iteration 
    results = np.array([])

    # Evaluate the gradient at the starting point 
    gradient_x = gradient(x0)

    # Initialize the steps counter 
    steps_count = 0

    # Set the initial point 
    x = x0 
    results = np.append(results, x, axis=0)

    # Iterate until the gradient is below the tolerance or maximum number of iterations is reached
    # Stopping criterion: inf norm of the gradient (max abs)
    while any(abs(gradient_x) > tolerance) and steps_count < max_iter:

        # Update the step size through the Armijo condition
        # Note: the first value of alpha is commonly set to 1
        #alpha = line_search(1, x, gradient_x)

        # Update the current point by moving in the direction of the negative gradient 
        x = x - alpha * gradient_x

        # Store the result
        results = np.append(results, x, axis=0)

        # Evaluate the gradient at the new point 
        gradient_x = gradient(x) 

        # Increment the iteration counter 
        steps_count += 1 

    # Return the steps taken and the number of steps
    return results.reshape(-1, 2), steps_count

In [ ]:
def LinearCG(A, b, x0, tol=1e-5):
    xk = x0
    rk = np.dot(A, xk) - b
    pk = -rk
    rk_norm = np.linalg.norm(rk)
    
    num_iter = 0
    curve_x = [xk]
    while rk_norm > tol:
        apk = np.dot(A, pk)
        rkrk = np.dot(rk, rk)
        
        alpha = rkrk / np.dot(pk, apk)
        xk = xk + alpha * pk
        rk = rk + alpha * apk
        beta = np.dot(rk, rk) / rkrk
        pk = -rk + beta * pk
        
        num_iter += 1
        curve_x.append(xk)
        rk_norm = np.linalg.norm(rk)
        print('Iteration: {} \t x = {} \t residual = {:.4f}'.
              format(num_iter, xk, rk_norm))
    
    print('\nSolution: \t x = {}'.format(xk))
        
    return np.array(curve_x)

In [ ]:
result = minimize(
    f, np.zeros(2), method='trust-constr', jac=df)

In [ ]:
def callbackF(Xi):
    global iterates
    iterates.append((Xi[0], Xi[1]))

In [ ]:
x0 = np.array([-9.0, -2.5])
iterates = [(x0[0], x0[1])]
cg_result = minimize(f, x0, method="cg", jac=df, callback=callbackF)
cg_result, iterates

In [ ]:
# Steepest descent
points, iters = steepest_descent(
  df, x0=x0, alpha=0.30)

# Found minimizer
minimizer = points[-1].round(1)

# Print results
print('- Final results: {}'.format(minimizer))
print('- N° steps: {}'.format(iters))

In [ ]:
# Prepare the objective function between -10 and 10
X, Y = np.meshgrid(np.linspace(-10, 10, 100), np.linspace(-10, 10, 100))
Z = f(np.array([X, Y]))

# Minimizer
min_x0, min_x1 = np.meshgrid(result.x[0], result.x[1])   
min_z = f(np.stack([min_x0, min_x1]))

# Prepare the objective function between -10 and 10
X_estimate, Y_estimate = points[:, 0], points[:, 1] 
Z_estimate = f(np.array([X_estimate, Y_estimate]))

iterates = np.array(iterates)
cg_X, cg_Y = iterates[:, 0], iterates[:, 1] 

In [ ]:
# Plot
with flopy.plot.styles.USGSPlot():
    fig = plt.figure(figsize=(4, 2), layout="constrained")
    
    # Second subplot
    ax = fig.add_subplot(1, 1, 1)
    ax.contour(X, Y, Z, 100, cmap='viridis', linewidths=0.5)
    # ax.plot(X_estimate, Y_estimate, Z_estimate, color='red', linewidth=3)
    # ax.scatter(min_x0, min_x1, min_z, marker='o', color='red', linewidth=10)
    ax.plot(X_estimate, Y_estimate, color='red', linewidth=1.25, marker="o", ms=2.5)
    ax.plot(cg_X, cg_Y, color='blue', linewidth=1.25, marker="o", ms=1.75)
    
    ax.plot(min_x0, min_x1, marker='o', mfc="none", mec="black", linewidth=0.0, ms=5)
    ax.plot(x0[0], x0[1], marker='s', color="black", linewidth=0.0, ms=5)
    ax.set_xlabel('$x_{0}$')
    ax.set_ylabel('$x_{1}$')
    ax.set_ylim(-5, 5)

    fig.savefig("figures/sd_cg_convergence.png", dpi=300, transparent=True)
    # ax.set_zlabel('$f(x)$')
    # ax.axes.zaxis.set_ticklabels([])